In [1]:
# %pylab inline

In [2]:
if False:
    plt.rcParams.update({'font.size': 16})
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')

In [3]:
import numpy as np
import itertools
import matplotlib as mpl

mpl.use('pgf')

import matplotlib.pyplot as plt 
plt.rcParams.update({
    "font.family": "serif",  # use serif/main font for text elements
    "text.usetex": True,     # use inline math for ticks
    "pgf.texsystem": "pdflatex",     # use inline math for ticks
    "pgf.rcfonts": False,    # don't setup fonts from rc parameters
    "pgf.preamble": "\n".join([
         r"\usepackage{times}",            # load additional packages
         r"\usepackage{bm}",            # load additional packages
    ])
})
import matplotlib.pyplot as plt

def proj_simplex(x):
    
    #x_sort = x[:]
    
    x_sort = np.sort(x)[::-1]
    acc = list(itertools.accumulate(x_sort))
    
    for i in range(len(x_sort)):
        val = x_sort[i] + (1 - acc[i])/(i+1)
        if (val <= 0):
            continue
        rho = i+1
        
    lam = (1 - acc[rho-1])/rho
    
    output = []
    
    for elem in x:
        val = max(0, elem + lam)
        output.append(val)
        
    return output

In [4]:
if True:
    # import numpy as np
    # import matplotlib.pyplot as plt
    import random

    fig = plt.figure(figsize=(10.5, 3))

    for it in range(3):
        
        x_p = np.array([random.uniform(0,1), random.uniform(0,1)])
        x_pp = np.array([random.uniform(0,1), random.uniform(0,1)])
        
        y_p = np.array([random.uniform(0,1), random.uniform(0,1)])
        y_pp = np.array([random.uniform(0,1), random.uniform(0,1)])
        
        lr = .1
        
        A = np.array([[1, -2],
                    [-1, 1]])
        
        B = np.array([[1, 1],
                    [1, -1]])
        
        B_T = B.transpose()
        
        #print(A, B)
        

        lst_x = []
        lst_y = []
        
        for i in range(800):
            x = x_p + 2*lr*A@y_p - lr*A@y_pp
            y = y_p + 2*lr*B_T@x_p - lr*B_T@x_pp

            lst_x.append(x[0])
            lst_y.append(y[0])
            
            x_pp, x_p = x_p, x
            y_pp, y_p = y_p, y
        
        plt.subplot(1, 3, it+1)
        
        plt.plot(lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        plt.xlabel('Iteration $t$')
        plt.plot(lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        plt.grid()
        plt.legend()
        
        
    # plt.show()
    fig.savefig("efficiency.pdf", bbox_inches="tight")
            

In [5]:
# import numpy as np
# import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10.5, 3))

for it in range(2):
    
    x_p = np.array([.5, .5])
    x_pp = np.array([.5, .5])
    
    y_p = np.array([5, .5])
    y_pp = np.array([.5, .5])
    
    eps = 0.05
    
    lr = .1
    
    A = np.array([[1, 0],
                [0, eps/2]])
    
    if (it == 1):
        B = np.array([[-1, 0],
                    [0, eps/2]])
    else:
        B = -A
    
    B_T = B.transpose()
    
    #print(A, B)
    

    xs = []
    lst_x = []
    lst_y = []

    if (it == 0):
        num_iter = 1000000
    else:
        num_iter = 1000
    
    for i in range(num_iter):
        x = x_p + 2*lr*A@y_p - lr*A@y_pp
        y = y_p + 2*lr*B_T@x_p - lr*B_T@x_pp

        if it == 0:
            if i % 4797 == 0:
                xs += [i]
                lst_x.append(x[1])
                lst_y.append(y[1])
        else:
            xs += [i]
            lst_x.append(x[1])
            lst_y.append(y[1])
        
        x_pp, x_p = x_p, x
        y_pp, y_p = y_p, y
    
    plt.subplot(1, 2, it+1)
    
    plt.plot(xs, lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(2)$")
    plt.xlabel('Iteration $t$')
    plt.plot(xs, lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(2)$")
    plt.grid()
    plt.legend()
    
    
# plt.show()
fig.savefig("robustness.pdf", bbox_inches="tight")

In [6]:
def predict(lst_util, method, H):
    if (lst_util == []):
        return np.array([0, 0, 0])
    if (method == "one-step"):
        return lst_util[-1]
    elif (method == "H-step"):
        return sum(lst_util[-H:])/H
    elif (method == "H-order"):
        if (H == 1):
            return lst_util[-1]
        elif (H == 2):
            return 2*lst_util[-1] - lst_util[-2]
        elif (H == 3):
            return 3*lst_util[-1] - 3*lst_util[-2] + lst_util[-3]
        elif (H == 4):
            return 4*lst_util[-1] - 6*lst_util[-2] + 4*lst_util[-3] - lst_util[-4]
    raise WTFError

In [7]:
# import numpy as np
# import matplotlib.pyplot as plt
# import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")
    else:
        fig.suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")

    num_iter = 2000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])

    A_T = A.transpose()
    
    for ep, H in enumerate([1, 10, 25, 50]):
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        pred_x = np.array([0, 0, 0])
        pred_y = np.array([0, 0, 0])
    
        lst_util_x = []
        lst_util_y = []
    
        lr = .05

        lst_x = []
        lst_y = []

        for i in range(num_iter):
            pred_x = predict(lst_util_x, method="H-step", H = H)
            x = proj_simplex(x_hat_p - lr*pred_x)

            pred_y = predict(lst_util_y, method="H-step", H = H)
            y = proj_simplex(y_hat_p + lr*pred_y)
            #y= y_hat_p*np.exp(lr*pred_y)
            #y/=sum(y)

            lst_x.append(x[0])
            lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            y_hat = proj_simplex(y_hat_p + lr*A_T@x)
            #y_hat = y_hat_p*np.exp(lr*A_T@x)
            #y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(A_T@x)
        #plt.plot(val_x)
        #plt.plot(val_y)

        # plt.subplot(3, 4, 4*it + ep+1)

        if (it == 0):
            axs[ep].set_title('$H=' + str(H) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iteration $t$')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')

        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
# plt.show()
# fig.tight_layout()
fig.savefig("H_step-both.pdf", bbox_inches="tight")

In [8]:
# import numpy as np
# import matplotlib.pyplot as plt
# import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")
    else:
        fig.suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")

    num_iter = 2000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])

    A_T = A.transpose()
    
    for ep, H in enumerate([1, 10, 25, 50]):
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        pred_x = np.array([0, 0, 0])
        pred_y = np.array([0, 0, 0])
    
        lst_util_x = []
        lst_util_y = []
    
        lr = .05

        lst_x = []
        lst_y = []

        for i in range(num_iter):
            pred_x = predict(lst_util_x, method="one-step", H = H)
            x = proj_simplex(x_hat_p - lr*pred_x)

            pred_y = predict(lst_util_y, method="H-step", H = H)
            y = proj_simplex(y_hat_p + lr*pred_y)
            #y= y_hat_p*np.exp(lr*pred_y)
            #y/=sum(y)

            lst_x.append(x[0])
            lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            y_hat = proj_simplex(y_hat_p + lr*A_T@x)
            #y_hat = y_hat_p*np.exp(lr*A_T@x)
            #y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(A_T@x)
        #plt.plot(val_x)
        #plt.plot(val_y)

        # plt.subplot(3, 4, 4*it + ep+1)

        if (it == 0):
            axs[ep].set_title('$H=' + str(H) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iteration $t$')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')

        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
# plt.show()
# fig.tight_layout()
fig.savefig("H_step-one.pdf", bbox_inches="tight")

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Bimatrix Game $(\\mathbf{{A}}_{it+1}, \\mathbf{{B}}_{it+1})$")
    else:
        fig.suptitle(f"Bimatrix Game $(\\mathbf{{A}}_{it+1}, \\mathbf{{B}}_{it+1})$")

    
    num_iter = 10000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
        v = np.array([-0.5, 0, 0.5])
        lam = .5
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
        v = np.array([-0.2, .5, -0.2])
        lam = 0.5
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])
        v = np.array([.5, .5, .5])
        lam = .2

        
    B = - lam*A + np.outer(np.ones(3), v)
    B_T = B.transpose()
    
    print(B)
    
    for ep, H in enumerate([1, 10, 25, 50]):
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        pred_x = np.array([0, 0, 0])
        pred_y = np.array([0, 0, 0])
    
        lst_util_x = []
        lst_util_y = []
    
        lr = .05

        lst_x = []
        lst_y = []
        xs = []
        for i in range(num_iter):
            pred_x = predict(lst_util_x, method="H-step", H = H)
            x = proj_simplex(x_hat_p - lr*pred_x)

            pred_y = predict(lst_util_y, method="H-step", H = H)
            y = proj_simplex(y_hat_p - lr*pred_y)

            #y= y_hat_p*np.exp(lr*pred_y)
            #y/=sum(y)

            if i % 173 == 0:
                xs += [i]
                lst_x.append(x[0])
                lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            y_hat = proj_simplex(y_hat_p - lr*B_T@x)
            #y_hat = y_hat_p*np.exp(lr*A_T@x)
            #y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(B_T@x)

        #plt.plot(val_x)
        #plt.plot(val_y)

        if (it == 0):
            axs[ep].set_title('$H=' + str(H) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iteration $t$')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')
        
        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(xs, lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(xs, lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
plt.show()
fig.savefig("strat-H_step-both.pdf", bbox_inches="tight")

[[-1.    0.5   1.  ]
 [ 0.    0.5   0.5 ]
 [-0.25  0.    1.  ]]
[[ 0.3   0.    0.3 ]
 [-0.2   0.25  0.3 ]
 [-0.35  0.75  0.05]]
[[0.7  0.5  0.56]
 [0.4  0.4  0.7 ]
 [0.5  0.6  0.5 ]]


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")
    else:
        fig.suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")

    
    num_iter = 10000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])
    A_T = A.transpose()

    A_T = A.transpose()
    
    for ep, delta in enumerate([.5, .8, .95, .96]):
        
        nom_x = np.array([0, 0, 0])
        denom_x = 0
        nom_y = np.array([0,0,0])
        denom_y = 0
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        lst_util_x = []
        lst_util_y = []
    
        lr = .02

        lst_x = []
        lst_y = []
        xs = []

        for i in range(num_iter):
            #pred_x = predict(lst_util_x, method="H-step", H = H)
            
            if (lst_util_x != []):
                nom_x = delta*nom_x + lst_util_x[-1]
                denom_x = delta*denom_x + 1
                pred_x = nom_x/denom_x
            else:
                pred_x = np.array([0, 0, 0])
            
            x = proj_simplex(x_hat_p - lr*pred_x)

            #pred_y = predict(lst_util_y, method="H-step", H = H)
            
            if (lst_util_y != []):
                nom_y = delta*nom_y + lst_util_y[-1]
                denom_y = delta*denom_y + 1
                pred_y = nom_y/denom_y
            else:
                pred_y = np.array([0, 0, 0])
            
            y = proj_simplex(y_hat_p + lr*pred_y)
            #y= y_hat_p*np.exp(lr*pred_y)
            #y/=sum(y)

            if i % 173 == 0:
                xs += [i]
                lst_x.append(x[0])
                lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            y_hat = proj_simplex(y_hat_p + lr*A_T@x)
            #y_hat = y_hat_p*np.exp(lr*A_T@x)
            #y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(A_T@x)

        #plt.plot(val_x)
        #plt.plot(val_y)


        if (it == 0):
            axs[ep].set_title('$\delta=' + str(delta) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iterations')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')
        
        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(xs, lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(xs, lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
plt.show()
fig.savefig("discounting-both.pdf", bbox_inches="tight")

In [11]:
# import numpy as np
# import matplotlib.pyplot as plt
# import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")
    else:
        fig.suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")

    num_iter = 2000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])

    A_T = A.transpose()
    
    for ep, H in enumerate([1, 2, 3, 4]):
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        pred_x = np.array([0, 0, 0])
        pred_y = np.array([0, 0, 0])
    
        lst_util_x = [pred_x] * 4
        lst_util_y = [pred_y] * 4
    
        lr = .05

        lst_x = []
        lst_y = []

        for i in range(num_iter):
            pred_x = predict(lst_util_x, method="H-order", H = H)
            x = proj_simplex(x_hat_p - lr*pred_x)

            pred_y = predict(lst_util_y, method="H-order", H = H)
            y = proj_simplex(y_hat_p + lr*pred_y)
            #y= y_hat_p*np.exp(lr*pred_y)
            #y/=sum(y)

            lst_x.append(x[0])
            lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            y_hat = proj_simplex(y_hat_p + lr*A_T@x)
            #y_hat = y_hat_p*np.exp(lr*A_T@x)
            #y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(A_T@x)
        #plt.plot(val_x)
        #plt.plot(val_y)

        # plt.subplot(3, 4, 4*it + ep+1)

        if (it == 0):
            axs[ep].set_title('$H=' + str(H) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iteration $t$')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')

        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
# plt.show()
# fig.tight_layout()
fig.savefig("H_order-both.pdf", bbox_inches="tight")

In [15]:
# import numpy as np
# import matplotlib.pyplot as plt
# import random

fig = plt.figure(constrained_layout=True, figsize=(8.9, 7))
figs = fig.subfigures(3,1)

for it in range(3):
    axs = figs[it].subplots(nrows=1, ncols=4, sharey=True)
    
    if it != 0:
        figs[it].suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")
    else:
        fig.suptitle(f"Matrix Game $(\\mathbf{{A}}_{it+1}, -\\mathbf{{A}}_{it+1})$")

    num_iter = 25000
    
    if (it == 0):
        A = np.array([[1, -1, -1],
                    [-1,-1, 0], 
                    [-.5, 0, -1]])
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])

    A_T = A.transpose()
    
    for ep, H in enumerate([1, 10, 25, 50]):
        
        x_hat_p = np.array([1/3, 1/3, 1/3])
        y_hat_p = np.array([1/3, 1/3, 1/3])
    
        pred_x = np.array([0, 0, 0])
        pred_y = np.array([0, 0, 0])
    
        lst_util_x = [pred_x] * 4
        lst_util_y = [pred_y] * 4
    
        lr = .05

        lst_x = []
        lst_y = []
        xs = []

        for i in range(num_iter):
            pred_x = predict(lst_util_x, method="H-step", H = H)
            x = proj_simplex(x_hat_p - lr*pred_x)

            pred_y = predict(lst_util_y, method="H-step", H = H)
            # y = proj_simplex(y_hat_p + lr*pred_y)
            y= y_hat_p*np.exp(lr*pred_y)
            y/=sum(y)

            if i % 373 == 0:
                xs += [i]
                lst_x.append(x[0])
                lst_y.append(y[0])

            x_hat = proj_simplex(x_hat_p - lr*A@y)
            # y_hat = proj_simplex(y_hat_p + lr*A_T@x)
            y_hat = y_hat_p*np.exp(lr*A_T@x)
            y_hat = y_hat/sum(y_hat)

            x_hat_p = x_hat
            y_hat_p = y_hat

            lst_util_x.append(A@y)
            lst_util_y.append(A_T@x)
        #plt.plot(val_x)
        #plt.plot(val_y)

        # plt.subplot(3, 4, 4*it + ep+1)

        if (it == 0):
            axs[ep].set_title('$H=' + str(H) + '$')
            
        if (it == 2):
            axs[ep].set_xlabel('Iteration $t$')
        
        if (ep == 0):
            axs[ep].set_ylabel('Action probability')

        #axs[ep].xlabel('Iterations')
        #axs[ep].ylabel('Probabilities')
        axs[ep].plot(xs, lst_x, color='#377eb8', label=r"$\bm{x}^{(t)}(1)$")
        axs[ep].plot(xs, lst_y, color='#ff7f00', ls="--", label=r"$\bm{y}^{(t)}(1)$")
        axs[ep].grid()

        if ep == 0:
            axs[ep].legend()
        
# plt.show()
# fig.tight_layout()
# fig.savefig("H_order-both.pdf", bbox_inches="tight")
fig.savefig("diff-H_step-both.pdf", bbox_inches="tight")

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import random

fig = plt.figure(figsize=(8, 4))

for it in range(1):
    
    num_iter = 100
    
    if (it == 0):
        A = np.array([[5, -1],
                    [-5,1]])
        B = np.array([[2, -2],
                    [-4, 4]])
        
    if (it == 1):
        A = np.array([[-1, 1, -1],
                        [0, .5, -1], 
                        [.3, -.5, -.5]])
    if (it == 2):
        A = np.array([[-1, 0, -.3],
                        [.5, .5, -1], 
                        [0, -.5, 0]])

    A_T = A.transpose()
    
    x_hat_p = np.array([1/2, 1/2])
    y_hat_p = np.array([1/2, 1/2])
    
    pred_x = np.array([0, 0])
    pred_y = np.array([0, 0])
    
        
    lst_util_x = []
    lst_util_y = []
    
    lr = .01

    lst_x = []
    lst_y = []

    for i in range(num_iter):
        
        if (lst_util_x == []):
            pred_x = np.array([0, 0])
        else:
            pred_x = lst_util_x[-1]
            
        x = proj_simplex(x_hat_p + lr*pred_x)

        if (lst_util_y == []):
            pred_y = np.array([0, 0])
        else:
            pred_y = lst_util_y[-1]
        
        y = proj_simplex(y_hat_p + lr*pred_y)
        #y= y_hat_p*np.exp(lr*pred_y)
        #y/=sum(y)

        lst_x.append(x)
        lst_y.append(y)

        x_hat = proj_simplex(x_hat_p + lr*A@y)
        y_hat = proj_simplex(y_hat_p + lr*A_T@x)
        #y_hat = y_hat_p*np.exp(lr*A_T@x)
        #y_hat = y_hat/sum(y_hat)

        x_hat_p = x_hat
        y_hat_p = y_hat

        lst_util_x.append(A@y)
        lst_util_y.append(A_T@x)

    #plt.plot(val_x)
    #plt.plot(val_y)

    plt.plot(lst_x)
    plt.plot(lst_y)
        
plt.show()
fig.savefig("potential.pdf", bbox_inches="tight")